<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PG24_2D_JSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2024-03-07 05:36:22.636757+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


#Install, Configure Local Database

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!pip install psycopg2
import psycopg2
import pandas as pd
pd.set_option('display.max_colwidth', 500)
#!sudo -u postgres psql -V
!psql -V
!service postgresql start





psql (PostgreSQL) 14.11 (Ubuntu 14.11-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


## Local Configuration

In [3]:
# This has created an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis
# Create file to store the password. Else will be prompted each time
#!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:upraxis:upass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:postgres:pass" >> ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass


ALTER ROLE
ALTER ROLE
localhost:5432:dbpraxis:upraxis:upass
localhost:5432:dbpraxis:postgres:pass


In [4]:
# Local Colab server

conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.11 (Ubuntu 14.11-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [5]:
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

#Local Database

## Create Tables

In [6]:
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop schema if exists js cascade'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'create schema if not exists js'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\dn+'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop table if exists js.orders'
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c '\
CREATE TABLE js.orders ( \
	id serial NOT NULL PRIMARY KEY, \
	info jsonb NOT NULL \
);'

#Describe the table ss_order
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c '\d+ js.orders'

NOTICE:  schema "js" does not exist, skipping
DROP SCHEMA
CREATE SCHEMA
                          List of schemas
  Name  |  Owner   |  Access privileges   |      Description       
--------+----------+----------------------+------------------------
 js     | upraxis  |                      | 
 public | postgres | postgres=UC/postgres+| standard public schema
        |          | =UC/postgres         | 
(2 rows)

NOTICE:  table "orders" does not exist, skipping
DROP TABLE
CREATE TABLE
                                                           Table "js.orders"
 Column |  Type   | Collation | Nullable |                Default                | Storage  | Compression | Stats target | Description 
--------+---------+-----------+----------+---------------------------------------+----------+-------------+--------------+-------------
 id     | integer |           | not null | nextval('js.orders_id_seq'::regclass) | plain    |             |              | 
 info   | jsonb   |           | not n

## Load Data - Insert Option

In [7]:
cmd = """INSERT INTO js.orders (info) VALUES('{ "customer": "Kapil Dev", "items": {"product": "Heineken","qty": 6}}');"""
#print(cmd)
runCmd(cmd)
runSelect('select * from js.orders')

Command executed


id,info
1,"{'items': {'qty': 6, 'product': 'Heineken'}, 'customer': 'Kapil Dev'}"


In [8]:
cmd = """INSERT INTO js.orders (info) \
VALUES('{ "customer": "Rekha Ganesan", "items": {"product": "Carlsberg","qty": 24}}'), \
      ('{ "customer": "Madhuri Dikshit", "items": {"product": "Kalyani","qty": 12}}'), \
      ('{ "customer": "Farokh Engineer", "items": {"product": "Kalyani Black","qty": 12}}'); \
"""
#print(cmd)
runCmd(cmd)
runSelect('select * from js.orders')

Command executed


id,info
1,"{'items': {'qty': 6, 'product': 'Heineken'}, 'customer': 'Kapil Dev'}"
2,"{'items': {'qty': 24, 'product': 'Carlsberg'}, 'customer': 'Rekha Ganesan'}"
3,"{'items': {'qty': 12, 'product': 'Kalyani'}, 'customer': 'Madhuri Dikshit'}"
4,"{'items': {'qty': 12, 'product': 'Kalyani Black'}, 'customer': 'Farokh Engineer'}"


In [9]:
cmd = """\
INSERT INTO js.orders (info) \
VALUES('{   "customer": "Saina Nehwal", \
            "items": [\
                {"product": "Bira91","qty": 6}, \
                {"product": "Kingfisher Light","qty": 6} \
            ] \
        }');\
"""
#print(cmd)
runCmd(cmd)
runSelect('select * from js.orders')

Command executed


id,info
1,"{'items': {'qty': 6, 'product': 'Heineken'}, 'customer': 'Kapil Dev'}"
2,"{'items': {'qty': 24, 'product': 'Carlsberg'}, 'customer': 'Rekha Ganesan'}"
3,"{'items': {'qty': 12, 'product': 'Kalyani'}, 'customer': 'Madhuri Dikshit'}"
4,"{'items': {'qty': 12, 'product': 'Kalyani Black'}, 'customer': 'Farokh Engineer'}"
5,"{'items': [{'qty': 6, 'product': 'Bira91'}, {'qty': 6, 'product': 'Kingfisher Light'}], 'customer': 'Saina Nehwal'}"


## Load Data - Copy Option

In [10]:
%%writefile data.json
{ "customer": "Kapil Dev", "items": {"product": "Heineken","qty": 6}}
{ "customer": "Rekha Ganesan", "items": {"product": "Carlsberg","qty": 24}}
{ "customer": "Madhuri Dikshit", "items": {"product": "Kalyani","qty": 12}}
{ "customer": "Farokh Engineer", "items": {"product": "Kalyani Black","qty": 12}}

Writing data.json


In [ ]:
#!cat data.json

In [11]:
# Only superuser ('postgres') can use copy command, not even database owner ('upraxis')
# 'postgres' password has to be entered manually
#
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "truncate js.orders ;"
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY js.orders(info) FROM '/content/data.json' ;"
#!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY empl FROM '/content/emp.csv' DELIMITER ',' CSV HEADER;"

TRUNCATE TABLE
COPY 4


In [12]:
# Data deleted by serial number NOT reset
runSelect('select * from js.orders')

id,info
6,"{'items': {'qty': 6, 'product': 'Heineken'}, 'customer': 'Kapil Dev'}"
7,"{'items': {'qty': 24, 'product': 'Carlsberg'}, 'customer': 'Rekha Ganesan'}"
8,"{'items': {'qty': 12, 'product': 'Kalyani'}, 'customer': 'Madhuri Dikshit'}"
9,"{'items': {'qty': 12, 'product': 'Kalyani Black'}, 'customer': 'Farokh Engineer'}"


In [13]:
%%writefile data.json
{ "customer": "Kapil", "items": {"product": "Heineken","qty": 6}}
{ "customer": "Satyen", "items": {"product": "Heineken","qty": 18}}
{ "customer": "Rekha", "items": {"product": "Carlsberg","qty": 24}}
{ "customer": "Madhuri", "items": {"product": "Kalyani","qty": 12}}
{ "customer": "Srinivas", "items": {"product": "Kingfisher Strong","qty": 12}}
{ "customer": "Saina", "items": [{"product": "Bira91","qty": 6},{"product": "Kalyani","qty": 6} ]}

Overwriting data.json


In [14]:
# Only superuser ('postgres') can use copy command, not even database owner ('upraxis')
# 'postgres' password has to be entered manually
#
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "truncate js.orders restart identity;"
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY js.orders(info) FROM '/content/data.json' ;"
#!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY empl FROM '/content/emp.csv' DELIMITER ',' CSV HEADER;"

TRUNCATE TABLE
COPY 6


In [15]:
runSelect('select * from js.orders')

id,info
1,"{'items': {'qty': 6, 'product': 'Heineken'}, 'customer': 'Kapil'}"
2,"{'items': {'qty': 18, 'product': 'Heineken'}, 'customer': 'Satyen'}"
3,"{'items': {'qty': 24, 'product': 'Carlsberg'}, 'customer': 'Rekha'}"
4,"{'items': {'qty': 12, 'product': 'Kalyani'}, 'customer': 'Madhuri'}"
5,"{'items': {'qty': 12, 'product': 'Kingfisher Strong'}, 'customer': 'Srinivas'}"
6,"{'items': [{'qty': 6, 'product': 'Bira91'}, {'qty': 6, 'product': 'Kalyani'}], 'customer': 'Saina'}"


# Access Data

In [16]:
#!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "select * from dept;"
cmd = "select * from js.orders;"
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "$cmd"

 id |                                                info                                                 
----+-----------------------------------------------------------------------------------------------------
  1 | {"items": {"qty": 6, "product": "Heineken"}, "customer": "Kapil"}
  2 | {"items": {"qty": 18, "product": "Heineken"}, "customer": "Satyen"}
  3 | {"items": {"qty": 24, "product": "Carlsberg"}, "customer": "Rekha"}
  4 | {"items": {"qty": 12, "product": "Kalyani"}, "customer": "Madhuri"}
  5 | {"items": {"qty": 12, "product": "Kingfisher Strong"}, "customer": "Srinivas"}
  6 | {"items": [{"qty": 6, "product": "Bira91"}, {"qty": 6, "product": "Kalyani"}], "customer": "Saina"}
(6 rows)



## Where conditions

In [17]:
cmd = "SELECT info ->> 'customer' AS customer FROM js.orders WHERE info -> 'items' ->> 'product' = 'Heineken'; "
#print(cmd)
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "$cmd"

 customer 
----------
 Kapil
 Satyen
(2 rows)



In [18]:
# ERROR

cmd = "SELECT info ->> 'customer' AS customer FROM js.orders WHERE info -> 'items' ->> 'product' = 'Kalyani'; "
#print(cmd)
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "$cmd"

 customer 
----------
 Madhuri
(1 row)



In [19]:
# ERROR CORRECTED

cmd = """SELECT id, info ->> 'customer' AS customer \
FROM   js.orders \
WHERE  info -> 'items' @? 'lax $.product ? (@ == \"Kalyani\")';"""
print(cmd)
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c """$cmd"""
runSelect(cmd)

SELECT id, info ->> 'customer' AS customer FROM   js.orders WHERE  info -> 'items' @? 'lax $.product ? (@ == "Kalyani")';


id,customer
4,Madhuri
6,Saina


In [20]:
# https://stackoverflow.com/questions/77772487/where-condition-to-filter-json-data-in-nested-object-or-array/

!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c """\
SELECT id, info ->> 'customer' AS customer \
FROM   js.orders \
WHERE  info -> 'items' @? '$.product ? (@ == \"Kalyani\")';\
"""


 id | customer 
----+----------
  4 | Madhuri
  6 | Saina
(2 rows)



# Enter MONGO DB

## Install Mongo

In [21]:
# Updated Install Sequence for MongoDB
# https://www.mongodb.com/docs/manual/tutorial/install-mongodb-on-ubuntu/
# Note Ubuntu versio 'jammy'
!curl -fsSL https://pgp.mongodb.com/server-6.0.asc | \
   sudo gpg -o /usr/share/keyrings/mongodb-server-6.0.gpg \
   --dearmor
# -----------------------
# Note reference to jammy
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list
# -----------------------
!sudo apt-get update > /dev/null
!sudo apt-get install -y mongodb-org > /dev/null              # ignore error and warning messages

deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 9.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [22]:
#!sudo service mongodb start --- this should work but does not
# -----------------------
# https://unix.stackexchange.com/questions/715119/system-v-init-service-cant-recongnise-mongod
!sudo mkdir -p /var/log/mongodb /var/lib/mongodb
!sudo chown `whoami` /var/lib/mongodb /var/log/mongodb/
# -----------------------
# Manually start MongoDB in background mode, to free up the terminal
!nohup mongod --config /etc/mongod.conf &
!mongod --version

nohup: appending output to 'nohup.out'
db version v6.0.14
Build Info: {
    "version": "6.0.14",
    "gitVersion": "25225db95574916fecab3af75b184409f8713aef",
    "openSSLVersion": "OpenSSL 3.0.2 15 Mar 2022",
    "modules": [],
    "allocator": "tcmalloc",
    "environment": {
        "distmod": "ubuntu2204",
        "distarch": "x86_64",
        "target_arch": "x86_64"
    }
}


In [23]:
!pip -qq install pymongo
from pymongo import MongoClient
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() # should show ['admin', 'config', 'local']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.0 MB/s eta 0:00:00


['admin', 'config', 'local']

## Create Database obhjects

In [24]:
# Delete a database
client.drop_database('PraxisDB')
# Create a new collection. We can create the collection or leave to MongoDB to create it as soon as a document is generated.
db = client.PraxisDB
db.create_collection('BeerBuyers')       # Optional collection creation
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
list (db.list_collections())
# empty list '[]' means that there are not collections in database

[{'name': 'BeerBuyers',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'\x1d8\x00\xbf\xb3{I`\x866\xb9\xba\xa4D\xe9\r', 4)},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

### Loading Data

In [25]:
cDoc1 =  {"customer": "Kapil", "items": {"product": "Heineken","qty": 6}}

In [26]:
kollection = db.BeerBuyer

In [28]:
insert_result = kollection.insert_one( cDoc1)
insert_result.acknowledged    # Confirms that insert is successful

True

### Retrieving Data

In [30]:
c = kollection.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('65e909108c26d94b094ba5c8'), 'customer': 'Kapil', 'items': {'product': 'Heineken', 'qty': 6}}


In [31]:
cDoc2 =  { "customer": "Satyen", "items": {"product": "Heineken","qty": 18}}
cDoc3 =  { "customer": "Rekha", "items": {"product": "Carlsberg","qty": 24}}
cDoc4 =  { "customer": "Madhuri", "items": {"product": "Kalyani","qty": 12}}
cDoc5 =  { "customer": "Srinivas", "items": {"product": "Kingfisher Strong","qty": 12}}
cDoc6 =  { "customer": "Saina", "items": [{"product": "Bira91","qty": 6},{"product": "Kalyani","qty": 6} ]}

cDocs = [cDoc2,cDoc3,cDoc4,cDoc5,cDoc6]

In [32]:
insert_result = kollection.insert_many(cDocs)

In [33]:
c = kollection.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('65e909108c26d94b094ba5c8'), 'customer': 'Kapil', 'items': {'product': 'Heineken', 'qty': 6}}
{'_id': ObjectId('65e915f18c26d94b094ba5c9'), 'customer': 'Satyen', 'items': {'product': 'Heineken', 'qty': 18}}
{'_id': ObjectId('65e915f18c26d94b094ba5ca'), 'customer': 'Rekha', 'items': {'product': 'Carlsberg', 'qty': 24}}
{'_id': ObjectId('65e915f18c26d94b094ba5cb'), 'customer': 'Madhuri', 'items': {'product': 'Kalyani', 'qty': 12}}
{'_id': ObjectId('65e915f18c26d94b094ba5cc'), 'customer': 'Srinivas', 'items': {'product': 'Kingfisher Strong', 'qty': 12}}
{'_id': ObjectId('65e915f18c26d94b094ba5cd'), 'customer': 'Saina', 'items': [{'product': 'Bira91', 'qty': 6}, {'product': 'Kalyani', 'qty': 6}]}


In [49]:
c1 = kollection.find({"items.product":"Kalyani"},{"customer"})
for crnt in c1:
    print(crnt)

{'_id': ObjectId('65e915f18c26d94b094ba5cb'), 'customer': 'Madhuri'}
{'_id': ObjectId('65e915f18c26d94b094ba5cd'), 'customer': 'Saina'}


In [52]:
c1 = kollection.find({"items.product":"Kalyani"},{"_id":0,"customer":1})
for crnt in c1:
    print(crnt)

{'customer': 'Madhuri'}
{'customer': 'Saina'}


In [54]:
c1 = kollection.find({"items.product":"Kalyani"},{"_id":0,"customer":1,"items":1})
for crnt in c1:
    print(crnt)

{'customer': 'Madhuri', 'items': {'product': 'Kalyani', 'qty': 12}}
{'customer': 'Saina', 'items': [{'product': 'Bira91', 'qty': 6}, {'product': 'Kalyani', 'qty': 6}]}


In [56]:
c1 = kollection.find(
    {"items.product":"Kalyani",
     "items.qty":{"$lt":10}},
    {"_id":0,"customer":1,"items":1})
for crnt in c1:
    print(crnt)

{'customer': 'Saina', 'items': [{'product': 'Bira91', 'qty': 6}, {'product': 'Kalyani', 'qty': 6}]}


In [57]:
c1 = kollection.find(
    {"items.product":"Kalyani",
     "items.qty":{"$gt":10}},
    {"_id":0,"customer":1,"items":1})
for crnt in c1:
    print(crnt)

{'customer': 'Madhuri', 'items': {'product': 'Kalyani', 'qty': 12}}


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)